In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import re

# Create a simple spam dataset
def create_spam_dataset():
    messages = [
        {"message": "Congratulations! You've won a free prize. Claim now!", "label": "spam"},
        {"message": "Hey, can we meet at 5pm today for coffee?", "label": "ham"},
        {"message": "URGENT: Your account has been suspended. Reply immediately.", "label": "spam"},
        {"message": "Don't forget to bring the documents for tomorrow's meeting.", "label": "ham"},
        {"message": "FREE entry to concert this weekend! Limited tickets!", "label": "spam"},
        {"message": "I'll be home late tonight, please feed the dog.", "label": "ham"},
        {"message": "Buy now! 80% off on all products. Limited time offer.", "label": "spam"},
        {"message": "The project deadline has been extended to next Friday.", "label": "ham"},
        {"message": "Click here to claim your inheritance from a distant relative.", "label": "spam"},
        {"message": "The doctor's appointment is confirmed for Thursday at 2pm.", "label": "ham"},
        {"message": "Win a new iPhone! Just fill out this quick survey.", "label": "spam"},
        {"message": "Can you pick up some groceries on your way home?", "label": "ham"},
    ]
    return pd.DataFrame(messages)

# Create dataset
df = create_spam_dataset()

# Display the dataset
print("Message Dataset:")
for i, row in df.iterrows():
    print(f"{i+1}. [{row['label'].upper()}] {row['message']}")

# Create features using bag of words approach
vectorizer = CountVectorizer()
X = vectorizer.fit_transform(df['message'])
y = df['label']

# Show feature names (words)
feature_names = vectorizer.get_feature_names_out()
print("\nVocabulary (unique words):")
print(feature_names)

# Show document-term matrix
X_array = X.toarray()
df_bow = pd.DataFrame(X_array, columns=feature_names)
print("\nBag of Words representation:")
print(df_bow)

# Create and train the Naive Bayes model
model = MultinomialNB()
model.fit(X, y)

# Print prior probabilities
print("\nPrior Probabilities:")
print(f"P(spam) = {model.class_count_[0]/sum(model.class_count_):.4f}")
print(f"P(ham) = {model.class_count_[1]/sum(model.class_count_):.4f}")

# Get feature (word) probabilities
feature_prob = pd.DataFrame({
    'Feature': feature_names,
    'Spam Probability': model.feature_log_prob_[0],
    'Ham Probability': model.feature_log_prob_[1]
})

# Convert from log probabilities to actual probabilities
feature_prob['Spam Probability'] = np.exp(feature_prob['Spam Probability'])
feature_prob['Ham Probability'] = np.exp(feature_prob['Ham Probability'])

# Add a ratio column to see which words are most indicative of spam
feature_prob['Spam/Ham Ratio'] = feature_prob['Spam Probability'] / feature_prob['Ham Probability']

# Sort by ratio to see most spam-indicative words
spam_words = feature_prob.sort_values('Spam/Ham Ratio', ascending=False)
print("\nTop spam-indicative words:")
print(spam_words.head(10))

# Sort to see most ham-indicative words
ham_words = feature_prob.sort_values('Spam/Ham Ratio')
print("\nTop ham-indicative words:")
print(ham_words.head(10))

# Visualize word probabilities
plt.figure(figsize=(12, 6))
# Select top N words from each class
top_n = 10
top_spam_features = spam_words.head(top_n)['Feature'].values
top_ham_features = ham_words.head(top_n)['Feature'].values

# Combine unique features
top_features = set(top_spam_features) | set(top_ham_features)
top_features = list(top_features)[:15]  # Limit to 15 words

# Get probabilities for selected features
selected_probs = feature_prob[feature_prob['Feature'].isin(top_features)]
selected_probs = selected_probs.sort_values('Spam/Ham Ratio', ascending=False)

# Plot
x = range(len(selected_probs))
width = 0.35
plt.bar([i - width/2 for i in x], selected_probs['Spam Probability'], width, label='Spam', color='red', alpha=0.7)
plt.bar([i + width/2 for i in x], selected_probs['Ham Probability'], width, label='Ham', color='green', alpha=0.7)
plt.xticks(x, selected_probs['Feature'], rotation=45)
plt.ylabel('Probability')
plt.title('Word Probabilities in Spam vs. Ham')
plt.legend()
plt.tight_layout()
plt.show()

# Test on new messages
test_messages = [
    "Congratulations! You won a free vacation to Hawaii!",
    "Hey, do you want to go to the movies tonight?",
    "URGENT: Your bank account needs verification immediately",
    "Please bring the report to the meeting tomorrow",
    "FREE GIFT waiting for you! Click now to claim!"
]

# Function to explain prediction
def explain_prediction(message, model, vectorizer):
    # Transform the message
    X_test = vectorizer.transform([message])

    # Get prediction and probability
    prediction = model.predict(X_test)[0]
    probs = model.predict_proba(X_test)[0]

    # Get word counts for this message
    words = X_test.toarray()[0]
    word_counts = []
    for i, count in enumerate(words):
        if count > 0:
            word = vectorizer.get_feature_names_out()[i]
            # Get probabilities for this word
            spam_prob = np.exp(model.feature_log_prob_[0][i])
            ham_prob = np.exp(model.feature_log_prob_[1][i])
            word_counts.append((word, count, spam_prob, ham_prob, spam_prob/ham_prob))

    # Sort by spam/ham ratio
    word_counts.sort(key=lambda x: x[4], reverse=True)

    # Print results
    print(f"\nMessage: {message}")
    print(f"Prediction: {prediction.upper()}")
    print(f"Confidence: {max(probs):.2%}")
    print(f"Probabilities: Spam={probs[0]:.2%}, Ham={probs[1]:.2%}")

    print("\nTop words influencing classification:")
    for word, count, spam_p, ham_p, ratio in word_counts[:5]:
        indicator = "→ SPAM" if spam_p > ham_p else "→ HAM"
        print(f"  {word}: {indicator} (Spam: {spam_p:.4f}, Ham: {ham_p:.4f}, Ratio: {ratio:.2f})")

# Test the classifier
print("\n=== Testing the classifier on new messages ===")
for message in test_messages:
    explain_prediction(message, model, vectorizer)